#README.txt

Drag and drop a ZIPPED version of the folder `/data` from GitHub repo into Colab.

< - - - - -

In [1]:
# then run this and refresh directory...

# unzip datasets
!unzip data.zip

Archive:  data.zip
   creating: data/
   creating: data/job-postings/
   creating: data/resumes/
  inflating: data/README.txt         
  inflating: __MACOSX/data/._README.txt  
  inflating: data/job-postings/Data_Job_TX.csv  
  inflating: data/job-postings/Data_Job_WA.csv  
  inflating: data/job-postings/Data_Job_NY.csv  
  inflating: data/job-postings/Data_Job_SF.csv  
  inflating: data/resumes/kaggleResumes.csv  
  inflating: __MACOSX/data/resumes/._kaggleResumes.csv  




---


# API

In [2]:
!pip install config

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import requests
import os
import config
# API Docs found here: https://developer.usajobs.gov/Tutorials/Search-Jobs

In [4]:
config.US_JOBS_API_KEY = "xVc0TZiLfhcr17ci7Ngk6bLAetdRVFgntm2pZgWNtww="
config.EMAIL_ADDRESS = "gjacobthomas@gmail.com"

Note: I'm assuming this API has some limiter on it so we don't want to lose access. -tyler

In [5]:
# more parameters are found here: https://developer.usajobs.gov/API-Reference/GET-api-Search
def queryJobsAPI(keyword, location):
  host = 'data.usajobs.gov' 
  # add these values in the config.py file
  userAgent = config.EMAIL_ADDRESS
  authKey = config.US_JOBS_API_KEY

  base_url = "https://data.usajobs.gov/api/search"

  parameters = {
      "Keyword": keyword,
      "LocationName": location
  }

  headers = {
      "Host": host,          
      "User-Agent": userAgent,          
      "Authorization-Key": authKey  
  }

  resp = requests.request("GET", base_url,headers=headers, params=parameters)
  result = resp.json()['SearchResult']['SearchResultItems']

  # get Job Title 
  print(result[1]['MatchedObjectDescriptor']['PositionTitle'])
  # get Job Summary
  print(result[1]['MatchedObjectDescriptor']['UserArea']['Details']['JobSummary'])
  return result




---


# KeyBERT Extraction Function

In [6]:
!pip install keybert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.1 MB/s 
     |████████████████████████████████| 237 kB 45.9 MB/s 
     |████████████████████████████████| 51 kB 7.0 MB/s 
     |████████████████████████████████| 5.5 MB 44.6 MB/s 
     |████████████████████████████████| 1.3 MB 35.2 MB/s 
     |████████████████████████████████| 182 kB 39.5 MB/s 
     |████████████████████████████████| 7.6 MB 22.7 MB/s 
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23800 sha256=4e2c8757f5f218de3775d08c3c20ef1edca4731c617dbb2f7a248b47977d1fc3
  Stored in directory: /root/.cache/pip/wheels/85/0d/12/77d219f3ebbb22dc22234b4d665886c0eace86a26eca0aa72b
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=cbcab2ff21c19e1b23e6f7ebc48aa5ef3bb19c73ac09626c01afb8597c23c6ae
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59

In [7]:
# Imports
from keybert import KeyBERT # pip install keybert (give it a minute...)

'''
/*---------------------------------------------------------------------
 |  Method: extractKeywordsBERT
 |
 |  Purpose: Uses the KeyBert Keyword Extraction Tool to extract
 |           and return keywords from a given corpus. 
 |      
 |  Author: Tyler Parks
 |  Created On: 10/30/22
 |
 |  Parameters:
 |      normalized_corpus -- A single string containing all text of the
 |                           normalized corpus.
 |
 |  Returns: 
 |      keywords -- List of collected keywords
 |      scores -- List of those keyword's scores
 |
 |  References: https://maartengr.github.io/KeyBERT/#usage
 |
 *-------------------------------------------------------------------*/
''' 
def extractKeywordsBERT(normalized_corpus):   
    print('---KeyBert Extraction---')
    print('------------------------\n')

    # init. language model 
    language_model = KeyBERT(model = 'all-mpnet-base-v2')

    # extract those keywords!
    data = language_model.extract_keywords( normalized_corpus, 
                                            keyphrase_ngram_range=(1, 3), 
                                            stop_words='english',
                                            use_maxsum=False, 
                                            use_mmr=True,
                                            diversity=0.7,
                                            nr_candidates=20, 
                                            top_n=15
                                        )

    # zip the lists
    zipped = list(map(list, zip(*data)))
    keywords = zipped[0]
    scores = zipped[1]

    print('-Skill-'.ljust(40), '-Score-')
    for i, value in enumerate(keywords):
        print(value.ljust(40), scores[i])
    print()

    return keywords, scores



---


# CSO-Classifier Extraction Function

In [8]:
!python -m spacy download en_core_web_sm
!pip install cso-classifier

2022-11-29 16:48:01.956724: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 5.2 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 3.2 MB 9.8 MB/s 
     |████████████████████████████████| 61 kB 7.1 MB/s 
     |████████████████████████████████| 1.5 MB 50.4 MB/s 
     |████████████████████████████████| 45 kB 3.1 MB/s 
     |████████████████████████████████| 50 kB 6.4 MB/s 
     |████████████████████████████████| 12.8 MB 43.9 MB/s 
     |████████████████████████████████| 24.2 MB 53.8 MB/s 
     |██████████████████

**Don't restart runtime if the terminal says so! Keep going.**

In [9]:
import nltk
nltk.download('stopwords')
# update spacy
!pip install -U spacy
import spacy
from cso_classifier import CSOClassifier      # import classifier tool

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.4 MB 5.3 MB/s 
     |████████████████████████████████| 806 kB 21.3 MB/s 
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.17
    Uninstalling thinc-8.0.17:
      Successfully uninstalled thinc-8.0.17
  Attempting uninstall: spacy
    Found existing installation: spacy 3.0.5
    Uninstalling spacy-3.0.5:
      Successfully uninstalled spacy-3.0.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cso-classifier 3.0 requires spacy==3.0.5, but you have spacy 3.4.3 which is incompatible.


In [10]:
# update the most recent model
CSOClassifier.update() 

# define the model object
CSO_Extractor = CSOClassifier(modules = "both", enhancement = "first", explanation = False)


# ======================================================
#     ONTOLOGY
# ======================================================
Updating the ontology file
[██████████████████████████████████████████████████] 7M/7M
[*] Done!
Extracting and converting ontology.
Creating ontology pickle file from a copy of the CSO Ontology found in /usr/local/lib/python3.7/dist-packages/cso_classifier/assets/cso.csv
Creating graph representation of the ontology.
Saving the graph representation of the ontology (in a pickle object) in /usr/local/lib/python3.7/dist-packages/cso_classifier/assets/cso_graph.p

# ======================================================
#     MODELS: CACHED & WORD2VEC
# ======================================================
Couldn't delete file cached model: not found
Couldn't delete file word2vec model: not found
Updating the models: cached and word2vec
[██████████████████████████████████████████████████] 63M/63M
[*] Done!
[██████████████████████████████████████████████████] 34

In [11]:
def extractKeywordsCSO(normalized_corpus):

  # run the extraction
  result = CSO_Extractor.run(normalized_corpus)

  print('-----CSO Extraction-----')
  print('------------------------\n')
  
  for keyword in result['union']:
    print(keyword)

  return result['union']

# **YAKE Extractor**

In [12]:
!pip3 install yake
import yake 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60 kB 3.2 MB/s 
     |████████████████████████████████| 132 kB 10.2 MB/s 
  Created wheel for jellyfish: filename=jellyfish-0.9.0-cp37-cp37m-linux_x86_64.whl size=73980 sha256=a76f8ad4a6b67b126b2c4840fe39490a36bf3873e0fdeccb10f11700a3b5c2e0
  Stored in directory: /root/.cache/pip/wheels/fe/99/4e/646ce766df0d070b0ef04db27aa11543e2767fda3075aec31b
Successfully built jellyfish


In [13]:
def extractKeywordsYAKE(normalized_corpus):
    print('---YAKE Extraction---')
    print('------------------------\n')

    language = "en"
    max_ngram_size = 3
    deduplication_threshold = 0.9
    deduplication_algo = 'seqm'
    windowSize = 1
    numOfKeywords = 20

    kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
    keywords = kw_extractor.extract_keywords(normalized_corpus)

    zipped = list(map(list, zip(*keywords)))
    keywords = zipped[0]
    scores = zipped[1]

    print('-Skill-'.ljust(40), '-Score-')
    for i, value in enumerate(keywords):
        print(value.ljust(40), scores[i])
    
    return keywords, scores

# **Levenstein Distance Matching**

In [14]:
!pip install jellyfish # the jellyfish library is used to calculate the Levenstein distance of two strings.
!pip install gensim
!pip install jieba
import jellyfish
from functools import reduce
import gensim
import nltk
from nltk.tokenize import word_tokenize
import jieba
nltk.download('punkt')
import re
import numpy as np

class SkillsMatching:
  def __init__(self, all_job_postings_keywords, all_jp_titles, all_resumes_keywords):
      self.all_job_postings_keywords = all_job_postings_keywords
      self.all_jp_titles = all_jp_titles
      self.all_resumes_keywords = all_resumes_keywords
      

  # output JSON with matching job postings for each resume
  def skill_match(self):
      results = {"matching_job": [],
                 "job_title": []}
      for resume_json in self.all_resumes_keywords:
          # need to change this to match all text classifiers
          resume_keywords = resume_json["keywordsBERT"]
          scores = self.sim_score(self.all_job_postings_keywords, resume_keywords)
          # grab the index of the largest score in the arr
          max_idx = np.argmax(scores)
          results['matching_job'].append(self.all_job_postings_keywords[max_idx])
          results['job_title'].append(self.all_jp_titles[max_idx])

      return results

  def split_and_join_arr(self, arr):
    new_arr = []
    for w in arr:
      word_arr = re.split('\W+', w.lower())
      new_arr = new_arr + word_arr
    # print(new_arr)
    return new_arr
  
  def sim_score(self, docs, keywords):
    keywords = self.split_and_join_arr(keywords)
    gen_docs = [[w.lower() for w in word_tokenize(text)] 
            for text in docs]
    dictionary = gensim.corpora.Dictionary(gen_docs)
    corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]
    tf_idf = gensim.models.TfidfModel(corpus)
    sims = gensim.similarities.Similarity('/usr/workdir',tf_idf[corpus],
                                      num_features=len(dictionary))

    query_doc_bow = dictionary.doc2bow(keywords)
    query_doc_tf_idf = tf_idf[query_doc_bow]
    # print(sims[query_doc_tf_idf])
    return sims[query_doc_tf_idf]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.




---


#Driver Code

In [15]:
# Imports
import pandas as pd     # pip install pandas. usage: loading data from csv files into dataframes
import nltk
from nltk.tokenize import word_tokenize  
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download("all")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

True

In [19]:
### Helper Functions

# Function to retrieve text data.
# (either 1 or more job postings or resumes)?
def getFileData(filename, dir):
    return pd.read_csv('data/' + dir + '/' + filename)

# Function to normalize text data. 
# (some skill extraction tools will normalize text for us; however, if not, this function is here)
# includes removing stopwords, punctuation, dates, links, etc...
def normalizeCorpus(corpus):
    return corpus
    # need to fix this 
    # nltk_tokenList = word_tokenize(corpus)
    # lemmatizer = WordNetLemmatizer()
    # nltk_lemmaList = []
    # for word in nltk_tokenList:
    #     nltk_lemmaList.append(lemmatizer.lemmatize(word))

    # normalized_corpus = []  
    # nltk_stop_words = set(stopwords.words("english"))
    # for w in nltk_lemmaList:  
    #     if w not in nltk_stop_words:  
    #         normalized_corpus.append(w)

    # punctuation = ";:.,?!"
    # for word in normalized_corpus:
    #     if word in punctuation:
    #         normalized_corpus.remove(word)

    #still need to add dates and links
    # return normalized_corpus

# Function to extract skill words from a given corpus.
# ideally, this function will output a set of skills extracted from the corpus
def extractSkills(corpus):

    # run KeyBert Extraction
    keywordsBERT, scoresBERT = extractKeywordsBERT(corpus)

    # run CSO Classifier Extraction
    keywordsCSO              = extractKeywordsCSO(corpus)
    
    # extraction method 2
    keywordsYAKE, scoresYAKE = extractKeywordsYAKE(corpus)
    # extraction method n...
    # keep going!
    return {"keywordsBERT": keywordsBERT,
            "keywordsCSO": keywordsCSO,
            "keywordsYAKE": keywordsYAKE}

### Driver Code
if __name__ == '__main__':

    # fetch the data
    job_posting_obj = queryJobsAPI("Data Scientist", "Washington, DC")
    resume_dataframe = getFileData('kaggleResumes.csv', 'resumes'     )
    #---------------

    '''
    # print the dataframes
    print('DataFrame of Job Postings:')
    print(job_posting_dataframe)    
    print()

    print('DataFrame of Resumes:')
    print(resume_dataframe)
    print()
    #----------------------
    '''

    # fetch the job descriptions and resumes by themselves
    jpCorpus = []
    jpTitles = []
    for i in range(len(job_posting_obj)):
      jpCorpus.append(job_posting_obj[i]['MatchedObjectDescriptor']['UserArea']['Details']['JobSummary'])
      jpTitles.append(job_posting_obj[i]['MatchedObjectDescriptor']['PositionTitle'])

    rCorpus  = list(resume_dataframe['Resume'])
    #----------------------

    # Number of resume samples to view
    NUM_SAMPLES = 10
    # number of job postings to view
    NUM_JPS = 100

    # for each JOB POSTING from the corpus
    i = 0
    all_job_postings = []
    for posting in jpCorpus:
        print('Job Posting #', i+1)
        print()

        text = normalizeCorpus(posting)
        job_posting_json = extractSkills(text)
        all_job_postings.append(text)
        # print lines, we are done with this posting
        print('------------------------\n')

        # break, after X postings
        i += 1
        if i > NUM_JPS:
            break
    #---------------------- 

    # for each RESUME from the corpus
    i = 0
    all_resumes_keywords = []
    for resume in rCorpus:
        print('Resume #', i+1)
        print()

        text = normalizeCorpus(resume)
        resume_json = extractSkills(text)
        all_resumes_keywords.append(resume_json)
        # print lines, we are done with this resume
        print('------------------------\n')

        # break, after X resumes
        i += 1
        if i > NUM_SAMPLES:
            break
    #---------------------- 

    
# keep going!

# end of driver code
#---------------

Data Scientist
DOE's Clean Energy Corps is made up of the staff from more than a dozen offices across DOE. As vacancies become available, applicants may be required to go through the DOE Clean Energy Corps hiring pipeline where you may be evaluated based on competencies through a core competency /technical interview(s). You are strongly encouraged to express interest through the Clean Energy Corps Applicant Portal for consideration. Failure to do so may result in loss of consideration for BIL positions.
Job Posting # 1

---KeyBert Extraction---
------------------------

-Skill-                                  -Score-
data scientist providing                 0.6331
dod workforce recruit                    0.5927
candidates positions competitive         0.3874
utilizing dha certain                    0.3603
appoint                                  0.3145
solicitation utilizing                   0.3052
certain personnel                        0.2893
work                                  

# Resume and Job Matching

In [20]:
# Resume and Job Posting Matching
matching = SkillsMatching(all_job_postings, jpTitles, all_resumes_keywords)
match_job_postings = matching.skill_match()
print(match_job_postings)
for i, jp in enumerate(match_job_postings['matching_job']):
  print('------------------------\n')
  print("Matching Job Posting for Resume %s" % (i + 1))
  print(match_job_postings['job_title'][i])
  print(jp)

{'matching_job': ['The incumbent is responsible for providing high-level technical expertise on the aggregation, collation, and the analyzation of data from databases including, but not limited to the development, implementation and updating of data extraction queries, data mining and developing strategies, action plans, data queries modification and maintenance, quality control and validation of collected data.', 'This is a Direct Hire Authority (DHA) solicitation utilizing the DHA for Certain Personnel of the DoD Workforce to recruit and appoint qualified candidates to positions in the competitive service. About the Position: Put your mind to work! Do Data Science for complex analytical, mathematical, and statistical research that shapes operational requirements and provide options on how to implement data driven solutions for the Department of Army.', 'This position is located in the Office of the Director for the National Institute of Food and Agriculture (NIFA). The selectee is no